**Passo 1: Setup do Self-Querying**

In [1]:
import os
from pathlib import Path
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_classic.chains.query_constructor.base import AttributeInfo
from langchain_classic.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.query_constructors.qdrant import QdrantTranslator
from dotenv import load_dotenv

e:\01-projetos\11-work\11.34-engenharia-vetorial\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:

env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
loaded = load_dotenv(dotenv_path=env_path)  # True se carregou
print("Carregou .env:", loaded, "->", env_path)

# Verificar uma variável
# print("GOOGLE_API_KEY:", os.getenv("GOOGLE_API_KEY"))


Carregou .env: True -> E:\01-projetos\11-work\11.34-engenharia-vetorial\.env


In [3]:

# Setup
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
llm = ChatGoogleGenerativeAI(model="gemini-flash-latest", temperature=0)

# Dados de Exemplo (Metadata Ricos)
docs = [
    Document(page_content="Como usar TPUs no Vertex AI", metadata={"year": 2024, "author": "Google", "topic": "AI"}),
    Document(page_content="Migrando SQL para BigQuery", metadata={"year": 2022, "author": "Google", "topic": "Data"}),
    Document(page_content="Tutorial de Kubernetes (GKE)", metadata={"year": 2023, "author": "Community", "topic": "DevOps"}),
    Document(page_content="Novidades do Gemini 1.5", metadata={"year": 2024, "author": "Google", "topic": "AI"}),
]

# Indexar no Qdrant (Memória para este lab ser rápido)
vectorstore = QdrantVectorStore.from_documents(
    docs, 
    embeddings, 
    location=":memory:", 
    collection_name="google_docs"
)

**Passo 2: Configurando o "Tradutor" (Gemini)**

In [4]:
# Definir os metadados para o LLM entender
metadata_field_info = [
    AttributeInfo(name="year", description="Ano de publicação", type="integer"),
    AttributeInfo(name="topic", description="Tópico técnico (AI, Data, DevOps)", type="string"),
    AttributeInfo(name="author", description="Autor (Google ou Community)", type="string"),
]

# Criar o Retriever
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    "Documentação Técnica",
    metadata_field_info,
    structured_query_translator=QdrantTranslator(metadata_key="metadata"),
    verbose=True # IMPORTANTE: Para ver o filtro JSON gerado!
)

**Passo 3: Executando Queries**

In [5]:
def testar_query(pergunta):
    print(f"\n🤖 Usuário: '{pergunta}'")
    try:
        results = retriever.invoke(pergunta)
        print(f"🔍 Filtro Gerado (Ver logs acima)")
        print(f"📚 Docs Retornados: {len(results)}")
        for doc in results:
            print(f"   - {doc.page_content} ({doc.metadata})")
    except Exception as e:
        print(f"Erro: {e}")

# Teste 1: Filtro Simples
testar_query("Documentos sobre AI publicados em 2024")

# Teste 2: Filtro Composto
testar_query("Materiais do Google sobre Data ou AI")

# Teste 3: A pergunta 'impossível' para busca vetorial pura
# Sem filtro, ele traria o tutorial de Kubernetes pois 'Google' e 'Community' são vetorialmente próximos em contexto tech.
testar_query("Quero apenas documentos oficiais do Google, nada da comunidade")


🤖 Usuário: 'Documentos sobre AI publicados em 2024'
🔍 Filtro Gerado (Ver logs acima)
📚 Docs Retornados: 2
   - Como usar TPUs no Vertex AI ({'year': 2024, 'author': 'Google', 'topic': 'AI', '_id': '8cfd8aacf80f44c4b215575d277c1129', '_collection_name': 'google_docs'})
   - Novidades do Gemini 1.5 ({'year': 2024, 'author': 'Google', 'topic': 'AI', '_id': 'c1b9a0c29ef14850b8daa584a30e8c71', '_collection_name': 'google_docs'})

🤖 Usuário: 'Materiais do Google sobre Data ou AI'
🔍 Filtro Gerado (Ver logs acima)
📚 Docs Retornados: 3
   - Como usar TPUs no Vertex AI ({'year': 2024, 'author': 'Google', 'topic': 'AI', '_id': '8cfd8aacf80f44c4b215575d277c1129', '_collection_name': 'google_docs'})
   - Migrando SQL para BigQuery ({'year': 2022, 'author': 'Google', 'topic': 'Data', '_id': 'ca27365bc21549afb219945d8c1a0c68', '_collection_name': 'google_docs'})
   - Novidades do Gemini 1.5 ({'year': 2024, 'author': 'Google', 'topic': 'AI', '_id': 'c1b9a0c29ef14850b8daa584a30e8c71', '_collection_nam